<a href="https://colab.research.google.com/github/karjain/Anti_deepFake/blob/main/Predict.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
%cd /content/drive

Mounted at /content/drive
/content/drive


In [ ]:
import numpy as np
import tensorflow as tf
import os
import cv2
import dlib
from tensorflow import keras
import PIL.Image as Image

In [ ]:
model = tf.keras.models.load_model('/content/drive/MyDrive/Fake_dect/modelK.h5')

In [ ]:
def get_boundingbox(face, width, height, scale=1.3, minsize=None):
    """
    Expects a dlib face to generate a quadratic bounding box.
    :param face: dlib face class
    :param width: frame width
    :param height: frame height
    :param scale: bounding box size multiplier to get a bigger face region
    :param minsize: set minimum bounding box size
    :return: x, y, bounding_box_size in opencv form
    """
    x1 = face.left()
    y1 = face.top()
    x2 = face.right()
    y2 = face.bottom()
    size_bb = int(max(x2 - x1, y2 - y1) * scale)
    if minsize:
        if size_bb < minsize:
            size_bb = minsize
    center_x, center_y = (x1 + x2) // 2, (y1 + y2) // 2

    # Check for out of bounds, x-y top left corner
    x1 = max(int(center_x - size_bb // 2), 0)
    y1 = max(int(center_y - size_bb // 2), 0)
    # Check for too big bb size for given x, y
    size_bb = min(width - x1, size_bb)
    size_bb = min(height - y1, size_bb)

    return x1, y1, size_bb

In [ ]:
def predict_face(model, video_files_path, frame_rate=33 ):
    
  frames=[]
  prediction_list = []
  confidence_list = []
  final_pred=0
  frames_with_faces = []
  cropped_faces_list = []

  video_file_name = video_files_path.split('/')[-1]
  video_file_name_only = video_file_name.split('.')[0]

  reader = cv2.VideoCapture(video_files_path)
  
  # Face detector
  face_detector = dlib.get_frontal_face_detector()
  
  # Frame numbers and length of output video
  i=0
  c=1
  while reader.isOpened():
      _, image = reader.read()
      if image is None:
          #print('frame')
          break
      if c%frame_rate==1:
        frames.append(image)
  
        # Image size
        height, width = image.shape[:2]

        # . Detect with dlib
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        faces = face_detector(gray)
        #If image with face found
        if len(faces):
            for face in faces:

              image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
              frames_with_faces.append(image)

              # Face crop with dlib and bounding box scale enlargement
              x, y, size = get_boundingbox(face, width, height)
              cropped_face = image[y:y+size, x:x+size]
              cropped_faces_list.append(cropped_face)
              
              #Code to save imgaes with faces(optional)
              # img = pImage.fromarray(image, 'RGB')
              # image_name = video_file_name_only+"_preprocessed_"+str(i)+'.png'
              # image_path = os.path.join(settings.PROJECT_DIR, 'uploaded_images', image_name)
              # img.save(image_path)    

              #Code to save cropped image(optional)
              # img = pImage.fromarray(cropped_face, 'RGB')
              # image_name = video_file_name_only+"_cropped_faces_"+str(i)+'.png'
              # image_path = os.path.join(settings.PROJECT_DIR, 'uploaded_images', image_name)
              # img.save(image_path)
              
              #Resize to models trained image size
              cropped_face = cv2.resize(cropped_face,(224,224))
              
              #Normalize image before feeding it into model
              norm_image = cv2.normalize(cropped_face, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)

              input_arr = keras.preprocessing.image.img_to_array(norm_image)
              input_arr = np.array([input_arr])
              prediction = model.predict(input_arr)
              prediction_list.append(prediction[0][0])
              confidence_list.append(round(prediction[0][0], 1))
              i+=1
        else:
          print("No Face found")
          break
      c+=1
  reader.release()           
        
  print("prediction_list",prediction_list)
  confidence = np.mean(prediction_list)
  prediction_list = [round(pred,1) for pred in prediction_list]

  if np.sum(confidence) > 0.9:
    final_pred = "REAL"
  else:
    final_pred = "FAKE"
  
  print("final ", final_pred)
  return final_pred, frames_with_faces, cropped_faces_list, confidence

In [ ]:
final_pred, frames_with_faces, cropped_faces_list, confidence = predict_face(reload_model,'/content/drive/MyDrive/Fake_dect/test_videos/real/aelfnikyqj.mp4')